In [13]:
import numpy as np
from hmmlearn import hmm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

conclusion: 

Feature Selection: CCI, ADX, Chaikin A/D
Model Selection: best using SVC with poly kernel
Hyperparameter Tuning: d = 2 c = 0.3

In [2]:
'''
CONST
'''
FILEPATH_STOCK_PRICE = "./tr_eikon_eod_data.csv"
STOCK_NAMES = ['AAPL.O', 'MSFT.O', 'INTC.O', 'AMZN.O', 'GS.N']

In [3]:
''' 
UTILS
'''
def sequential_train_test_split(data, ratio = 0.8):
    df = pd.DataFrame(data.dropna()).to_numpy()
    split_index = int(len(df) * ratio)
    return df[:split_index], df[split_index:]


In [4]:
'''
BENCHMARK
'''

def benchmark_accuracy(prediction, test):
    prediction = pd.DataFrame(prediction)
    test = pd.DataFrame(test)

    return (prediction == test).sum() / len(prediction)


In [5]:
''' 
MODEL
'''

def predict_position(train, test):
    # Train Hidden Markov Model
    model = hmm.GaussianHMM(n_components=2, covariance_type="diag", n_iter=1000)
    model.fit(train.reshape(-1, 1))

    # Predict hidden states for test data
    hidden_states = model.predict(test.reshape(-1, 1))

    # Define a function to map hidden states to rise or fall
    def map_to_rise_or_fall(state):
        return 1 if state == 0 else -1

    # Map hidden states to rise or fall
    predicted_rise_fall = np.array(list(map(map_to_rise_or_fall, hidden_states)))

    return predicted_rise_fall

In [8]:
apple_data = pd.read_csv('all.csv', index_col = 0, parse_dates = True)
# apple_data['SMA_EMA_ratio'] = apple_data['SMA'] / apple_data['EMA']
apple_data['direction'] = np.where(apple_data['AAPL.O'] <= apple_data['AAPL.O'].shift(-1), 1, -1)

apple_data

/var/folders/ql/hnw16fjd1bb7227zpj_cw9140000gn/T/ipykernel_80941/2336444998.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  apple_data = pd.read_csv('all.csv', index_col = 0, parse_dates = True)


,AMZN.O,SlowK,SlowD,RSI,ADX,CCI,Aroon Down,Aroon Up,OBV,Chaikin A/D,...,RSI.4,ADX.4,CCI.4,Aroon Down.4,Aroon Up.4,OBV.4,Chaikin A/D.4,SMA.4,EMA.4,direction
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
2010-01-04,133.90,56.8287,73.2678,66.1726,35.5107,8.9870,57.1429,85.7143,1.140431e+10,2.070208e+10,...,50.3525,25.5682,-119.6258,100.0000,64.2857,-177810600.0,811806291.3,137.0128,135.9827,1
2010-01-05,134.69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
2010-01-06,132.25,75.6546,78.7052,45.0015,45.8330,12.7304,57.1429,14.2857,1.096472e+10,2.068413e+10,...,49.2538,31.4835,85.6424,71.4286,14.2857,-346418300.0,664477496.8,110.7708,112.1395,-1
2010-01-07,130.00,22.3903,20.1191,21.6477,46.2261,-114.8337,85.7143,21.4286,1.030262e+10,2.043025e+10,...,37.8543,33.0676,-128.0550,100.0000,71.4286,-430663700.0,621506911.0,107.1164,106.6629,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-25,1663.15,28.7844,51.2296,53.7404,46.1234,-197.7752,100.0000,85.7143,2.248574e+10,3.437684e+10,...,32.7234,22.7421,-152.1742,100.0000,42.8571,-240254198.0,643702133.9,201.4811,200.4351,1
2018-06-26,1691.09,25.7092,34.9854,44.5310,43.3708,-120.6630,92.8571,78.5714,2.257347e+10,3.441648e+10,...,28.5329,24.4674,-140.4348,100.0000,35.7143,-237561311.0,644609519.7,200.4011,199.2830,-1
2018-06-27,1660.51,21.6400,25.3779,54.7483,40.0760,-113.1170,85.7143,71.4286,2.247603e+10,3.432094e+10,...,28.0921,24.3184,-105.2860,92.8571,28.5714,-240664932.0,641552482.1,199.3360,198.2927,1


In [9]:
scaler = MinMaxScaler()

# Normalize the selected columns
normalized_columns = scaler.fit_transform(apple_data.iloc[:, :-1])

# Create a new DataFrame with the normalized values
normalized_df = pd.DataFrame(normalized_columns, columns=apple_data.columns[:-1])

# Display the normalized DataFrame
apple_data.iloc[:, :-1] = normalized_df
apple_data

,AMZN.O,SlowK,SlowD,RSI,ADX,CCI,Aroon Down,Aroon Up,OBV,Chaikin A/D,...,RSI.4,ADX.4,CCI.4,Aroon Down.4,Aroon Up.4,OBV.4,Chaikin A/D.4,SMA.4,EMA.4,direction
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
2010-01-04,0.015407,0.547956,0.743189,0.642768,0.470929,0.506203,0.571429,0.857143,0.125422,0.082504,...,0.475208,0.226894,0.311874,1.000000,0.642857,0.895429,0.940148,0.398379,0.389250,1
2010-01-05,0.015888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
2010-01-06,0.014402,0.745369,0.803009,0.359953,0.656789,0.511902,0.571429,0.142857,0.094772,0.081320,...,0.462212,0.311360,0.676722,0.714286,0.142857,0.453773,0.484560,0.234722,0.239687,-1
2010-01-07,0.013031,0.186826,0.158464,0.047981,0.663867,0.317693,0.857143,0.214286,0.048608,0.064585,...,0.327378,0.333980,0.296891,1.000000,0.714286,0.233098,0.351682,0.211932,0.205333,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-25,0.947041,0.253876,0.500732,0.476692,0.662018,0.191420,1.000000,0.857143,0.898056,0.983911,...,0.266690,0.186540,0.254021,1.000000,0.428571,0.731862,0.420316,0.800433,0.793548,1
2018-06-26,0.964063,0.221629,0.322018,0.353668,0.612456,0.308819,0.928571,0.785714,0.904173,0.986523,...,0.217124,0.211175,0.274887,1.000000,0.357143,0.738916,0.423122,0.793698,0.786321,-1
2018-06-27,0.945433,0.178958,0.216319,0.490156,0.553130,0.320307,0.857143,0.714286,0.897378,0.980226,...,0.211910,0.209048,0.337361,0.928571,0.285714,0.730786,0.413669,0.787055,0.780109,1


In [104]:
# apple_data = apple_data[['SlowK', 'RSI', 'CCI', 'Aroon Down', 'OBV', 'direction']]

In [10]:
train_data, test_data = sequential_train_test_split(apple_data)
X_train, y_train, X_test, y_test = train_data[:, :-1], train_data[:, -1], test_data[:, :-1], test_data[:, -1]

In [11]:
X_train

array([[0.01540692, 0.54795571, 0.74318886, ..., 0.94014795, 0.39837939,
        0.38925041],
       [0.01440173, 0.74536875, 0.8030094 , ..., 0.48456046, 0.23472223,
        0.23968656],
       [0.013031  , 0.18682613, 0.15846381, ..., 0.35168174, 0.21193172,
        0.20533289],
       ...,
       [0.4390394 , 0.22154924, 0.33421347, ..., 0.47464902, 0.42850586,
        0.43312621],
       [0.43518919, 0.19793421, 0.25432366, ..., 0.4703103 , 0.4347261 ,
        0.43762506],
       [0.429091  , 0.10654342, 0.17140289, ..., 0.46453625, 0.44006576,
        0.44023142]])

In [28]:

model = SVC(kernel='rbf', gamma=0.7, C=1.0)
model.fit(X_train, y_train)
pred = model.predict(X_test)
benchmark_accuracy(pred, y_test)

0    0.507937
dtype: float64

In [29]:
best_acc = 0
best_g = -1
best_c = -1

selected_features = [4, 5, 9]

for g in range(1, 11, 1):
    for c in range(1, 51, 1):
        curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
        curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
        model = SVC(kernel='rbf', gamma=g / 10, C=c / 10)
        model.fit(curr_X_train, y_train)
        pred = model.predict(curr_X_test)
        up_percentage = ((pred > 0).sum() / len(pred))
        acc = benchmark_accuracy(pred, y_test)
        print(f"acc: {round(acc[0], 3)} up: {round(up_percentage, 3)}")

        if acc[0] > best_acc:
            best_acc = acc[0]
            best_g = g
            best_c = c
        print(f"best acc: {best_acc}, d: {best_g}, c: {best_c}")
    
print(f"best acc: {best_acc}, g: {best_g}, c: {best_c}")

acc: 0.519 up: 0.0
best acc: 0.5185185185185185, d: 1, c: 1
acc: 0.519 up: 0.0
best acc: 0.5185185185185185, d: 1, c: 1
acc: 0.519 up: 0.0
best acc: 0.5185185185185185, d: 1, c: 1
acc: 0.519 up: 0.0
best acc: 0.5185185185185185, d: 1, c: 1
acc: 0.519 up: 0.0
best acc: 0.5185185185185185, d: 1, c: 1
acc: 0.521 up: 0.003
best acc: 0.5211640211640212, d: 1, c: 6
acc: 0.521 up: 0.003
best acc: 0.5211640211640212, d: 1, c: 6
acc: 0.519 up: 0.005
best acc: 0.5211640211640212, d: 1, c: 6
acc: 0.524 up: 0.016
best acc: 0.5238095238095238, d: 1, c: 9
acc: 0.526 up: 0.019
best acc: 0.5264550264550265, d: 1, c: 10
acc: 0.537 up: 0.045
best acc: 0.5370370370370371, d: 1, c: 11
acc: 0.545 up: 0.058
best acc: 0.544973544973545, d: 1, c: 12
acc: 0.548 up: 0.061
best acc: 0.5476190476190477, d: 1, c: 13
acc: 0.556 up: 0.079
best acc: 0.5555555555555556, d: 1, c: 14
acc: 0.561 up: 0.09
best acc: 0.5608465608465608, d: 1, c: 15
acc: 0.563 up: 0.098
best acc: 0.5634920634920635, d: 1, c: 16
acc: 0.553 up

KeyboardInterrupt: 

In [151]:
best_acc = 0
best_c = -1

selected_features = [4, 5, 9]

for c in range(1, 101, 1):
    curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
    curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
    model = SVC(kernel='sigmoid', C= c/10)
    model.fit(curr_X_train, y_train)
    pred = model.predict(curr_X_test)
    up_percentage = ((pred > 0).sum() / len(pred))
    acc = benchmark_accuracy(pred, y_test)
    print(f"acc: {round(acc[0], 3)} up: {round(up_percentage, 3)}")

    if acc[0] > best_acc:
        best_acc = acc[0]
        best_g = g
        best_c = c
    print(f"best acc: {best_acc}, c: {best_c}")
    
print(f"best acc: {best_acc}, c: {best_c}")

acc: 0.519 up: 0.0
best acc: 0.5185185185185185, c: 1
acc: 0.519 up: 0.0
best acc: 0.5185185185185185, c: 1
acc: 0.521 up: 0.198
best acc: 0.5211640211640212, c: 3
acc: 0.532 up: 0.098
best acc: 0.5317460317460317, c: 4
acc: 0.542 up: 0.743
best acc: 0.5423280423280423, c: 5
acc: 0.537 up: 0.749
best acc: 0.5423280423280423, c: 5
acc: 0.503 up: 0.037
best acc: 0.5423280423280423, c: 5
acc: 0.489 up: 0.071
best acc: 0.5423280423280423, c: 5
acc: 0.495 up: 0.082
best acc: 0.5423280423280423, c: 5
acc: 0.487 up: 0.101
best acc: 0.5423280423280423, c: 5
acc: 0.484 up: 0.108
best acc: 0.5423280423280423, c: 5
acc: 0.497 up: 0.132
best acc: 0.5423280423280423, c: 5
acc: 0.481 up: 0.148
best acc: 0.5423280423280423, c: 5
acc: 0.481 up: 0.159
best acc: 0.5423280423280423, c: 5
acc: 0.468 up: 0.172
best acc: 0.5423280423280423, c: 5
acc: 0.46 up: 0.18
best acc: 0.5423280423280423, c: 5
acc: 0.463 up: 0.188
best acc: 0.5423280423280423, c: 5
acc: 0.463 up: 0.204
best acc: 0.5423280423280423, c: 

In [136]:
best_acc = 0
best_d = -1
best_c = -1

selected_features = [54, ]

for d in range(1, 5, 1):
    for c in range(1, 51, 1):
        curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
        curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
        model = SVC(kernel='poly', degree=d, C=c/10)
        model.fit(curr_X_train, y_train)
        pred = model.predict(curr_X_test)
        up_percentage = ((pred > 0).sum() / len(pred))
        acc = benchmark_accuracy(pred, y_test)
        print(f"acc: {round(acc[0], 3)} up: {round(up_percentage, 3)}")

        if acc[0] > best_acc:
            best_acc = acc[0]
            best_d = d
            best_c = c
        print(f"best acc: {best_acc}, d: {best_d}, c: {best_c}")
    
print(f"best acc: {best_acc}, d: {best_d}, c: {best_c}")

acc: 0.595 up: 0.177
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.579 up: 0.246
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.577 up: 0.243
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.571 up: 0.259
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.571 up: 0.259
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.571 up: 0.259
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.571 up: 0.259
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.571 up: 0.259
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.571 up: 0.259
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.571 up: 0.259
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.571 up: 0.259
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.571 up: 0.259
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.571 up: 0.259
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.569 up: 0.262
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.571 up: 0.259
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.571 up: 0.259
best acc: 0.5952380952380952, d: 1, c: 1
acc: 0.5

In [23]:
best_acc = 0
best_c = -1

selected_features = [54, 27]

# for d in range(1, 5, 1):
for c in range(1, 51, 1):
    curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
    curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
    model = SVC(kernel='poly', degree=2, C = c / 10)
    model.fit(curr_X_train, y_train)
    pred = model.predict(curr_X_test)
    up_percentage = ((pred > 0).sum() / len(pred))
    acc = benchmark_accuracy(pred, y_test)
    print(f"acc: {round(acc[0], 3)} up: {round(up_percentage, 3)}")

    if acc[0] > best_acc:
        best_acc = acc[0]
        best_c = c
    print(f"best acc: {best_acc}, c: {best_c}")
    
print(f"best acc: {best_acc}, c: {best_c}")

acc: 0.537 up: 0.331
best acc: 0.5370370370370371, c: 1
acc: 0.537 up: 0.331
best acc: 0.5370370370370371, c: 1
acc: 0.537 up: 0.331
best acc: 0.5370370370370371, c: 1
acc: 0.537 up: 0.331
best acc: 0.5370370370370371, c: 1
acc: 0.537 up: 0.331
best acc: 0.5370370370370371, c: 1
acc: 0.537 up: 0.331
best acc: 0.5370370370370371, c: 1
acc: 0.537 up: 0.331
best acc: 0.5370370370370371, c: 1
acc: 0.537 up: 0.331
best acc: 0.5370370370370371, c: 1
acc: 0.537 up: 0.331
best acc: 0.5370370370370371, c: 1
acc: 0.537 up: 0.331
best acc: 0.5370370370370371, c: 1
acc: 0.537 up: 0.331
best acc: 0.5370370370370371, c: 1
acc: 0.537 up: 0.331
best acc: 0.5370370370370371, c: 1
acc: 0.537 up: 0.331
best acc: 0.5370370370370371, c: 1
acc: 0.537 up: 0.331
best acc: 0.5370370370370371, c: 1
acc: 0.537 up: 0.331
best acc: 0.5370370370370371, c: 1
acc: 0.537 up: 0.331
best acc: 0.5370370370370371, c: 1
acc: 0.537 up: 0.331
best acc: 0.5370370370370371, c: 1
acc: 0.537 up: 0.331
best acc: 0.537037037037037

In [ ]:
# Feature Engineering

# Feature Selection

In [43]:
X = apple_data.dropna().iloc[:, :-1]
y = apple_data.dropna().iloc[:, -1]

In [44]:
X

,AAPL.O,SlowK,SlowD,RSI,ADX,CCI,Aroon Down,Aroon Up,OBV,Chaikin A/D,SMA,EMA,SMA_EMA_ratio
Date,,,,,,,,,,,,,
2010-04-01,0.018837,0.797082,0.851941,0.769280,0.413634,0.658869,0.428571,1.000000,0.466485,0.197673,0.021918,0.021476,0.436146
2010-06-01,0.016229,0.907128,0.844247,0.536156,0.374250,0.748379,0.571429,1.000000,0.417465,0.386577,0.033742,0.034488,0.269045
2010-07-01,0.015895,0.111497,0.109400,0.287327,0.221484,0.249224,1.000000,0.428571,0.311572,0.333500,0.045891,0.041627,0.950701
2010-11-01,0.015491,0.258310,0.233819,0.445504,0.149730,0.329648,0.000000,0.285714,0.566643,0.506703,0.070674,0.068125,0.680743
2010-12-01,0.013441,0.723159,0.787620,0.558717,0.137354,0.660432,0.357143,0.000000,0.561133,0.530421,0.072551,0.072439,0.398836
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-25,0.929088,0.244971,0.261688,0.215016,0.272481,0.283454,1.000000,0.142857,0.572554,0.836229,0.812646,0.803231,0.652194
2018-06-26,0.942658,0.308398,0.263699,0.201690,0.271435,0.383573,0.928571,0.071429,0.576126,0.836040,0.808467,0.799351,0.645859
2018-06-27,0.941037,0.375016,0.287289,0.339535,0.260639,0.428531,0.857143,0.000000,0.572450,0.832650,0.804889,0.798193,0.587664


In [45]:
y

Date
2010-04-01    1
2010-06-01   -1
2010-07-01    1
2010-11-01   -1
2010-12-01    1
             ..
2018-06-25    1
2018-06-26   -1
2018-06-27    1
2018-06-28   -1
2018-06-29   -1
Name: direction, Length: 1887, dtype: int64

In [46]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Assuming 'X' contains the features and 'y' contains the target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Selecting the most relevant features using mutual information
selector = SelectKBest(score_func=mutual_info_classif, k=5)  # Select top 5 features
X_train_selected = selector.fit_transform(X_train, y_train)
selected_features = X.columns[selector.get_support()]

# Train a model using the selected features
model = LogisticRegression()
model.fit(X_train_selected, y_train)

# Evaluate the model on the test set
X_test_selected = selector.transform(X_test)
accuracy = model.score(X_test_selected, y_test)

# Display the most relevant features and model accuracy
print("Selected features:", selected_features)
print("Model accuracy:", accuracy)

Selected features: Index(['SlowK', 'RSI', 'CCI', 'OBV', 'EMA'], dtype='object')
Model accuracy: 0.5238095238095238


In [17]:
best_acc = 0

features = []

for i in range(len(apple_data.columns) - 1):
    curr_X_train = X_train[:, 1].reshape(-1, 1)
    curr_X_test = X_test[:, i].reshape(-1, 1)
    model = SVC(kernel='rbf', gamma= 1.0, C= 1.0)
    model.fit(curr_X_train, y_train)
    pred = model.predict(curr_X_test)
    up_percentage = ((pred > 0).sum() / len(pred))
    acc = benchmark_accuracy(pred, y_test)
    print(f"feature {i}: acc: {round(acc[0], 3)} up: {round(up_percentage, 3)}")
    features.append((acc[0], i))

features.sort(key = lambda i: -i[0])
print(features)

feature 0: acc: 0.487 up: 0.286
feature 1: acc: 0.484 up: 0.437
feature 2: acc: 0.505 up: 0.447
feature 3: acc: 0.489 up: 0.257
feature 4: acc: 0.481 up: 0.09
feature 5: acc: 0.489 up: 0.198
feature 6: acc: 0.526 up: 0.251
feature 7: acc: 0.495 up: 0.537
feature 8: acc: 0.484 up: 0.738
feature 9: acc: 0.481 up: 1.0
feature 10: acc: 0.487 up: 0.254
feature 11: acc: 0.487 up: 0.254
feature 12: acc: 0.463 up: 0.765
feature 13: acc: 0.537 up: 0.479
feature 14: acc: 0.529 up: 0.481
feature 15: acc: 0.505 up: 0.262
feature 16: acc: 0.532 up: 0.082
feature 17: acc: 0.553 up: 0.272
feature 18: acc: 0.516 up: 0.257
feature 19: acc: 0.503 up: 0.529
feature 20: acc: 0.519 up: 0.0
feature 21: acc: 0.481 up: 0.989
feature 22: acc: 0.468 up: 0.394
feature 23: acc: 0.474 up: 0.384
feature 24: acc: 0.479 up: 0.421
feature 25: acc: 0.532 up: 0.426
feature 26: acc: 0.513 up: 0.423
feature 27: acc: 0.55 up: 0.249
feature 28: acc: 0.505 up: 0.082
feature 29: acc: 0.542 up: 0.193
feature 30: acc: 0.5 up: 0

In [19]:
df_features = pd.DataFrame(features)
df_features

,0,1
0,0.555556,54
1,0.552910,17
2,0.550265,27
3,0.547619,43
4,0.542328,29
5,0.537037,13
6,0.537037,39
7,0.531746,16
8,0.531746,25
9,0.531746,57


In [14]:
best_acc = 0
best_selected_features = []

for i in range(len(apple_data.columns) - 1):
    for j in range(len(apple_data.columns) - 1):
        selected_features = [i, j]
        curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
        curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
        model = SVC(kernel='poly', degree = 2, C= 0.3)
        model.fit(curr_X_train, y_train)
        pred = model.predict(curr_X_test)
        up_percentage = ((pred > 0).sum() / len(pred))
        acc = benchmark_accuracy(pred, y_test)
        print(f"feature {i}: acc: {round(acc[0], 3)} up: {round(up_percentage, 3)}")

        if acc[0] > best_acc:
            best_acc = acc[0]
            best_selected_features = selected_features
        print(f"best acc: {best_acc} features: {best_selected_features}")

feature 0: acc: 0.519 up: 0.0
best acc: 0.5185185185185185 features: [0, 0]
feature 0: acc: 0.505 up: 0.103
best acc: 0.5185185185185185 features: [0, 0]
feature 0: acc: 0.516 up: 0.003
best acc: 0.5185185185185185 features: [0, 0]
feature 0: acc: 0.513 up: 0.265
best acc: 0.5185185185185185 features: [0, 0]
feature 0: acc: 0.519 up: 0.042
best acc: 0.5185185185185185 features: [0, 0]
feature 0: acc: 0.481 up: 0.307
best acc: 0.5185185185185185 features: [0, 0]
feature 0: acc: 0.548 up: 0.14
best acc: 0.5476190476190477 features: [0, 6]
feature 0: acc: 0.519 up: 0.032
best acc: 0.5476190476190477 features: [0, 6]
feature 0: acc: 0.463 up: 0.204
best acc: 0.5476190476190477 features: [0, 6]
feature 0: acc: 0.519 up: 0.0
best acc: 0.5476190476190477 features: [0, 6]
feature 0: acc: 0.519 up: 0.005
best acc: 0.5476190476190477 features: [0, 6]
feature 0: acc: 0.519 up: 0.005
best acc: 0.5476190476190477 features: [0, 6]
feature 0: acc: 0.519 up: 0.0
best acc: 0.5476190476190477 features: 

KeyboardInterrupt: 

In [26]:
ranked_features = [f for a, f in features]
ranked_features

[54,
 17,
 27,
 43,
 29,
 13,
 39,
 16,
 25,
 57,
 14,
 6,
 31,
 50,
 40,
 52,
 20,
 41,
 18,
 38,
 26,
 51,
 55,
 42,
 37,
 2,
 15,
 28,
 53,
 19,
 49,
 30,
 58,
 59,
 48,
 7,
 3,
 5,
 0,
 10,
 11,
 34,
 35,
 1,
 8,
 32,
 45,
 56,
 4,
 9,
 21,
 24,
 33,
 44,
 23,
 46,
 47,
 22,
 12,
 36]

In [27]:
import itertools

all_features = ranked_features
selected_features = list(itertools.permutations(all_features, 2))

best_acc = 0
best_selected_features = []

for i in range(1, len(apple_data.columns) - 1):
    possible_selected_features = list(itertools.permutations(all_features, i))
    for selected_features in possible_selected_features:
        curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
        curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
        model = SVC(kernel='poly', degree = 2, C= 0.3)
        model.fit(curr_X_train, y_train)
        pred = model.predict(curr_X_test)
        up_percentage = ((pred > 0).sum() / len(pred))
        acc = benchmark_accuracy(pred, y_test)
        print(f"feature {i}: acc: {round(acc[0], 3)} up: {round(up_percentage, 3)}, features: {selected_features}")

        if acc[0] > best_acc:
            best_acc = acc[0]
            best_selected_features = selected_features
        print(f"best acc: {best_acc} features: {best_selected_features}")

feature 1: acc: 0.519 up: 0.0, features: (54,)
best acc: 0.5185185185185185 features: (54,)
feature 1: acc: 0.534 up: 0.466, features: (17,)
best acc: 0.5343915343915344 features: (17,)
feature 1: acc: 0.532 up: 0.389, features: (27,)
best acc: 0.5343915343915344 features: (17,)
feature 1: acc: 0.548 up: 0.468, features: (43,)
best acc: 0.5476190476190477 features: (43,)
feature 1: acc: 0.55 up: 0.169, features: (29,)
best acc: 0.5502645502645502 features: (29,)
feature 1: acc: 0.545 up: 0.471, features: (13,)
best acc: 0.5502645502645502 features: (29,)
feature 1: acc: 0.537 up: 0.161, features: (39,)
best acc: 0.5502645502645502 features: (29,)
feature 1: acc: 0.532 up: 0.082, features: (16,)
best acc: 0.5502645502645502 features: (29,)
feature 1: acc: 0.519 up: 0.0, features: (25,)
best acc: 0.5502645502645502 features: (29,)
feature 1: acc: 0.521 up: 0.003, features: (57,)
best acc: 0.5502645502645502 features: (29,)
feature 1: acc: 0.526 up: 0.458, features: (14,)
best acc: 0.5502

KeyboardInterrupt: 